# Goldenspike - Interactive Version: an example of an end-to-end analysis using RAIL

**Authors:** Sam Schmidt, Eric Charles, Alex Malz, John Franklin Crenshaw, others...

**Last run successfully:** Dec 16, 2025

This notebook demonstrates how to use a the various RAIL Modules to draw synthetic samples of fluxes by color, apply physical effects to them, train photo-Z estimators on the samples, test and validate the preformance of those estimators, and to use the RAIL summarization modules to obtain n(z) estimates based on the p(z) estimates.

**Creation** 

Note that in the parlance of the Creation Module, "degradation" is any post-processing that occurs to the "true" sample generated by the create Engine.  This can include adding photometric errors, applying quality cuts, introducing systematic biases, etc. 

In this notebook, we will draw both test and training samples from a RAIL Engine object. Then we will demonstrate how to use RAIL degraders to apply effects to those samples.

**Training and Estimation** 

The RAIL Informer modules "train" or "inform" models used to estimate p(z) given band fluxes (and potentially other information).

The RAIL Estimation modules then use those same models to actually apply the model and extract the p(z) estimates.

**p(z) Validation** 

The RAIL Validator module applies various metrics.

**p(z) to n(z) Summarization** 

The RAIL Summarization modules convert per-galaxy p(z) posteriors to ensemble n(z) estimates. 

In [ ]:
import numpy as np
import tables_io
from pzflow.examples import get_galaxy_data

import rail.interactive as ri

Here we need a few configuration parameters to deal with differences in data schema between existing PZ codes.

In [ ]:
bands = ["u", "g", "r", "i", "z", "y"]
band_dict = {band: f"mag_{band}_lsst" for band in bands}
rename_dict = {f"mag_{band}_lsst_err": f"mag_err_{band}_lsst" for band in bands}

## Get the data to use

In [ ]:
catalog = get_galaxy_data().rename(band_dict, axis=1)

## Train the Flow Engine

First we need to train the normalizing flow that will serve as the engine for the notebook.

In [ ]:
flow_model = ri.creation.engines.flowEngine.flow_modeler(
    input=catalog,
    seed=0,
    phys_cols={"redshift": [0.3]},
    phot_cols={
        "mag_u_lsst": [17, 35],
        "mag_g_lsst": [16, 32],
        "mag_r_lsst": [15, 30],
        "mag_i_lsst": [15, 30],
        "mag_z_lsst": [14, 29],
        "mag_y_lsst": [14, 28],
    },
    calc_colors={"ref_column_name": "mag_i_lsst"},
)

## Make mock data

Now we will use the trained flow to create training and test data for the photo-z estimators.

For both the training and test data we will:

1. Use the Flow to produce some synthetic data
2. Use the LSSTErrorModel to add photometric errors
3. Use the FlowPosterior to estimate the redshift posteriors for the degraded sample
4. Use the ColumnMapper to rename the error columns so that they match the names in DC2.
5. Use the TableConverter to convert the data to a numpy dictionary, which will be stored in a hdf5 file with the same schema as the DC2 data

### Training sample

For the training data we are going to apply a couple of extra degradation effects to the data beyond what we do to create test data, as the training data will have some spectroscopic incompleteness.  This will allow us to see how the trained models perform with imperfect training data.

More details about the degraders are available in the `rail/examples/creation_examples/degradation_demo.ipynb` notebook.


In [ ]:
train_data_orig = ri.creation.engines.flowEngine.flow_creator(
    n_samples=150, model=flow_model["model"], seed=1235
)

In [ ]:
train_data_errs = ri.creation.degraders.photometric_errors.lsst_error_model(
    input=train_data_orig["output"], seed=66, renameDict=band_dict, ndFlag=np.nan
)

In [ ]:
train_data_inc = (
    ri.creation.degraders.spectroscopic_degraders.inv_redshift_incompleteness(
        input=train_data_errs["output"], pivot_redshift=1.0
    )
)

In [ ]:
train_data_conf = ri.creation.degraders.spectroscopic_degraders.line_confusion(
    input=train_data_inc["output"],
    true_wavelen=5007.0,
    wrong_wavelen=3727.0,
    frac_wrong=0.05,
    seed=1337,
)

In [ ]:
train_data_cut = ri.creation.degraders.quantityCut.quantity_cut(
    input=train_data_conf["output"], cuts={"mag_i_lsst": 25.0}
)

In [ ]:
train_data_pq = ri.tools.table_tools.column_mapper(
    input=train_data_cut["output"], columns=rename_dict
)

In [ ]:
train_data = ri.tools.table_tools.table_converter(
    input=train_data_pq["output"], output_format="numpyDict"
)

Let's examine the quantities that we've generated, we'll use the handy `tables_io` package to temporarily write to a pandas dataframe for quick writeout of the columns:

In [ ]:
train_table = tables_io.convertObj(train_data["output"], tables_io.types.PD_DATAFRAME)
train_table.head()

You see that we've generated redshifts, ugrizy magnitudes, and magnitude errors with names that match those in the cosmoDC2_v1.1.4_image data.

### Testing sample

For the test sample we will:

1. Use the Flow to produce some synthetic data
2. Use the LSSTErrorModel to smear the data
3. Use the FlowPosterior to estimate the redshift posteriors for the degraded sample
4. Use ColumnMapper to rename some of the columns to match DC2
5. Use the TableConverter to convert the data to a numpy dictionary, which will be stored in a hdf5 file with the same schema as the DC2 data

In [ ]:
test_data_orig = ri.creation.engines.flowEngine.flow_creator(
    model=flow_model["model"], n_samples=150, seed=1234
)

In [ ]:
test_data_errs = ri.creation.degraders.photometric_errors.lsst_error_model(
    input=test_data_orig["output"], seed=58, renameDict=band_dict, ndFlag=np.nan
)

In [ ]:
test_data_post = ri.creation.engines.flowEngine.flow_posterior(
    input=test_data_errs["output"],
    model=flow_model["model"],
    column="redshift",
    grid=np.linspace(0.0, 5.0, 21),
    err_samples=None,
)

In [ ]:
test_data_pq = ri.tools.table_tools.column_mapper(
    input=test_data_errs["output"], columns=rename_dict, hdf5_groupname=""
)

In [ ]:
test_data = ri.tools.table_tools.table_converter(
    input=test_data_pq["output"], output_format="numpyDict"
)

In [ ]:
test_table = tables_io.convertObj(test_data["output"], tables_io.types.PD_DATAFRAME)
test_table.head()

## "Inform" some estimators

More details about the process of "informing" or "training" the models used by the estimators is available in the `rail/examples/estimation_examples/RAIL_estimation_demo.ipynb` notebook.

We use "inform" rather than "train" to generically refer to the preprocessing of any prior information.
For a machine learning estimator, that prior information is a training set, but it can also be an SED template library for a template-fitting or hybrid estimator.

In [ ]:
inform_bpz = ri.estimation.algos.bpz_lite.bpz_lite_informer(
    input=train_data["output"], nondetect_val=np.nan, model="bpz.pkl", hdf5_groupname=""
)
inform_knn = ri.estimation.algos.k_nearneigh.k_near_neig_informer(
    input=train_data["output"], nondetect_val=np.nan, model="bpz.pkl", hdf5_groupname=""
)
inform_fzboost = ri.estimation.algos.flexzboost.flex_z_boost_informer(
    input=train_data["output"], nondetect_val=np.nan, model="bpz.pkl", hdf5_groupname=""
)

## Estimate photo-z posteriors

More detail on the specific estimators used here is available in the `rail/examples/estimation_examples/RAIL_estimation_demo.ipynb` notebook, but here is a very brief summary of the three estimators used in this notebook:

`BPZliteEstimator` is a template-based photo-z code that outputs the posterior estimated given likelihoods calculated using a template set combined with a Bayesian prior. See Benitez (2000) for more details.<br>
`KNearNeighEstimator` is a simple photo-z code that finds the K nearest neighbor training galaxies in color/magnitude space and creates a weighted (by distance) mixture model PDF based on the redshifts of those K neighbors.<br>
`FlexZBoostEstimator` is a mature photo-z algorithm that estimates a PDF for each galaxy via a conditional density estimate using the training data.  See [Izbicki & Lee (2017)](https://doi.org/10.1214/17-EJS1302) for more details.<br>


In [ ]:
bpz_estimated = ri.estimation.algos.bpz_lite.bpz_lite_estimator(
    input=test_data["output"],
    model=inform_bpz["model"],
    nondetect_val=np.nan,
    hdf5_groupname="",
)
knn_estimated = ri.estimation.algos.k_nearneigh.k_near_neig_estimator(
    input=test_data["output"],
    model=inform_knn["model"],
    nondetect_val=np.nan,
    hdf5_groupname="",
)
fzboost_estimated = ri.estimation.algos.flexzboost.flex_z_boost_estimator(
    input=test_data["output"],
    model=inform_fzboost["model"],
    nondetect_val=np.nan,
    hdf5_groupname="",
    aliases={"input": "test_data", "output": "fzboost_estim"},
)

## Evaluate the estimates

Now we evaluate metrics on the estimates, separately for each estimator.  

Each call to the `Evaluator.evaluate` will create a table with the various performance metrics. 
We will store all of these tables in a dictionary, keyed by the name of the estimator.

In [ ]:
eval_dict = dict(bpz=bpz_estimated, fzboost=fzboost_estimated, knn=knn_estimated)


evaluator_stage_dict = dict(
    metrics=["cdeloss", "pit", "brier"],
    _random_state=None,
    metric_config={
        "brier": {"limits": (0, 3.1)},
        "pit": {"tdigest_compression": 1000},
    },
)
truth = test_data_orig

result_dict = {}
for key, val in eval_dict.items():
    the_eval = ri.evaluation.dist_to_point_evaluator.dist_to_point_evaluator(
        input={"data": val["output"], "truth": truth["output"]},
        **evaluator_stage_dict,
        hdf5_groupname="",
    )
    result_dict[key] = the_eval

In [ ]:
result_dict

The Pandas DataFrame output format conveniently makes human-readable printouts of the metrics.  
This next cell will convert everything to Pandas.

In [ ]:
results_tables = {
    key: tables_io.convertObj(val["summary"], tables_io.types.PD_DATAFRAME)
    for key, val in result_dict.items()
}

In [ ]:
results_tables["knn"]

In [ ]:
results_tables["fzboost"]

In [ ]:
results_tables["bpz"]

## Summarize the per-galaxy redshift constraints to make population-level distributions

{introduce the summarizers}

First we make the stages, then execute them, then plot the output.

In [ ]:
point_estimate_ens = ri.estimation.algos.point_est_hist.point_est_hist_summarizer(
    input=eval_dict["bpz"]["output"]
)
naive_stack_ens = ri.estimation.algos.naive_stack.naive_stack_summarizer(
    input=eval_dict["bpz"]["output"]
)

In [ ]:
point_estimate_ens["output"].plot_native(xlim=(0, 3))

In [ ]:
naive_stack_ens["output"].plot_native(xlim=(0, 3))